In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from models_def import cnn2d,cnn_v1,cnn_v2,cudnn_gru,lstm_v1,cnn_gru,gru_v1
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import text, sequence

model_defines = [
#     [cnn2d,'cnn2d'],
#     [cnn_v1,'cnn_v1'],
#     [cnn_v2,'cnn_v2'],
    [cudnn_gru,'cudnn_gru'],
    [lstm_v1,'lstm_v1'],
    [cnn_gru,'cnn_gru'],
#    [gru_v1,'gru_v1']
]

max_features = 180000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(159571, 6)


In [2]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
# 
word_vec_dict = {}
with open('../crawl-300d-2M.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.rstrip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')

EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

(159571, 150) (153164, 150)
plunder 35873
chic 26355
aprox 44391
cloverfield 16859
vonnegut 73231
swaggart 61347
2000000
Preparing embedding matrix
Null word embeddings: 79399


In [3]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
from keras import backend as K
import pickle
from keras.models import load_model
import gc

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def kf_train(model_func,fold_cnt=3,rnd=1,epo=10,batch=64):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = model_func(maxlen,nb_words,EMBEDDING_DIM,embedding_matrix,trainable_flag=False,comp=True)
        batch_size = batch
        epochs = epo
        file_path="best_model.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
        
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    eval_val(y,train_pred)
    return train_pred, test_pred

rnd_init = 1
for model_func,model_name in model_defines:
    for fold_k in [5]:
        print('==================================')
        epo = 10
        batch = 64
        train_pred,test_pred = kf_train(model_func,fold_k,rnd_init,epo,batch)
        output_feat = '../features/fasttext_{}_{}_feat.pkl'.format(model_name,fold_k)
        with open(output_feat,'wb') as fout:
            pickle.dump([train_pred,test_pred],fout)
        print(output_feat,'done')
        rnd_init += 1
        




Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 177s 1ms/step - loss: 0.0529 - acc: 0.9803 - val_loss: 0.0441 - val_acc: 0.9833
Epoch 2/10
127656/127656 [==============================] - 177s 1ms/step - loss: 0.0432 - acc: 0.9832 - val_loss: 0.0490 - val_acc: 0.9805
Epoch 3/10
127656/127656 [==============================] - 178s 1ms/step - loss: 0.0417 - acc: 0.9838 - val_loss: 0.0421 - val_acc: 0.9836
Epoch 4/10
127656/127656 [==============================] - 178s 1ms/step - loss: 0.0386 - acc: 0.9847 - val_loss: 0.0412 - val_acc: 0.9839
Epoch 5/10
127656/127656 [==============================] - 180s 1ms/step - loss: 0.0363 - acc: 0.9852 - val_loss: 0.0406 - val_acc: 0.9842
Epoch 6/10
127656/127656 [==============================] - 176s 1ms/step - loss: 0.0343 - acc: 0.9861 - val_loss: 0.0425 - val_acc: 0.9838
Epoch 7/10
127656/127656 [==============================] - 177s 1ms/step - loss: 0.0320 - acc: 0.9868 - val_

Epoch 10/10
127657/127657 [==============================] - 173s 1ms/step - loss: 0.0334 - acc: 0.9862 - val_loss: 0.0440 - val_acc: 0.9838
Train on 127657 samples, validate on 31914 samples
Epoch 1/10
127657/127657 [==============================] - 175s 1ms/step - loss: 0.0493 - acc: 0.9816 - val_loss: 0.0425 - val_acc: 0.9833
Epoch 2/10
127657/127657 [==============================] - 176s 1ms/step - loss: 0.0405 - acc: 0.9841 - val_loss: 0.0399 - val_acc: 0.9843
Epoch 3/10
127657/127657 [==============================] - 175s 1ms/step - loss: 0.0372 - acc: 0.9851 - val_loss: 0.0400 - val_acc: 0.9842
Epoch 4/10
127657/127657 [==============================] - 175s 1ms/step - loss: 0.0339 - acc: 0.9863 - val_loss: 0.0410 - val_acc: 0.9840
Epoch 5/10
127657/127657 [==============================] - 175s 1ms/step - loss: 0.0310 - acc: 0.9872 - val_loss: 0.0432 - val_acc: 0.9837
Epoch 6/10
127657/127657 [==============================] - 175s 1ms/step - loss: 0.0269 - acc: 0.9887 - val

Epoch 8/10
127656/127656 [==============================] - 89s 695us/step - loss: 0.0318 - acc: 0.9867 - val_loss: 0.0418 - val_acc: 0.9837
Epoch 9/10
127656/127656 [==============================] - 88s 693us/step - loss: 0.0308 - acc: 0.9872 - val_loss: 0.0436 - val_acc: 0.9839
Epoch 10/10
127656/127656 [==============================] - 85s 668us/step - loss: 0.0298 - acc: 0.9877 - val_loss: 0.0442 - val_acc: 0.9839
Train on 127657 samples, validate on 31914 samples
Epoch 1/10
127657/127657 [==============================] - 89s 700us/step - loss: 0.0512 - acc: 0.9810 - val_loss: 0.0432 - val_acc: 0.9825
Epoch 2/10
127657/127657 [==============================] - 90s 708us/step - loss: 0.0416 - acc: 0.9836 - val_loss: 0.0397 - val_acc: 0.9844
Epoch 3/10
127657/127657 [==============================] - 87s 680us/step - loss: 0.0392 - acc: 0.9843 - val_loss: 0.0399 - val_acc: 0.9842
Epoch 4/10
127657/127657 [==============================] - 89s 701us/step - loss: 0.0374 - acc: 0.985

Epoch 7/10
127657/127657 [==============================] - 87s 681us/step - loss: 0.0330 - acc: 0.9864 - val_loss: 0.0401 - val_acc: 0.9842
Epoch 8/10
127657/127657 [==============================] - 88s 688us/step - loss: 0.0317 - acc: 0.9870 - val_loss: 0.0427 - val_acc: 0.9824
Epoch 9/10
127657/127657 [==============================] - 89s 698us/step - loss: 0.0308 - acc: 0.9873 - val_loss: 0.0424 - val_acc: 0.9836
Epoch 10/10
127657/127657 [==============================] - 89s 701us/step - loss: 0.0299 - acc: 0.9877 - val_loss: 0.0431 - val_acc: 0.9835
Train on 127657 samples, validate on 31914 samples
Epoch 1/10
127657/127657 [==============================] - 89s 694us/step - loss: 0.0508 - acc: 0.9809 - val_loss: 0.0430 - val_acc: 0.9834
Epoch 2/10
127657/127657 [==============================] - 90s 708us/step - loss: 0.0417 - acc: 0.9836 - val_loss: 0.0398 - val_acc: 0.9841
Epoch 3/10
127657/127657 [==============================] - 89s 694us/step - loss: 0.0391 - acc: 0.984

Epoch 5/10
127657/127657 [==============================] - 59s 458us/step - loss: 0.0316 - acc: 0.9872 - val_loss: 0.0424 - val_acc: 0.9837
Epoch 6/10
127657/127657 [==============================] - 59s 461us/step - loss: 0.0294 - acc: 0.9880 - val_loss: 0.0446 - val_acc: 0.9834
Epoch 7/10
127657/127657 [==============================] - 59s 461us/step - loss: 0.0279 - acc: 0.9886 - val_loss: 0.0445 - val_acc: 0.9831
Epoch 8/10
127657/127657 [==============================] - 60s 470us/step - loss: 0.0263 - acc: 0.9893 - val_loss: 0.0473 - val_acc: 0.9833
Epoch 9/10
127657/127657 [==============================] - 59s 464us/step - loss: 0.0250 - acc: 0.9898 - val_loss: 0.0465 - val_acc: 0.9826
Epoch 10/10
127657/127657 [==============================] - 59s 460us/step - loss: 0.0242 - acc: 0.9901 - val_loss: 0.0486 - val_acc: 0.9828
Train on 127657 samples, validate on 31914 samples
Epoch 1/10
127657/127657 [==============================] - 60s 470us/step - loss: 0.0505 - acc: 0.981

Epoch 5/10
127657/127657 [==============================] - 59s 461us/step - loss: 0.0319 - acc: 0.9871 - val_loss: 0.0432 - val_acc: 0.9834
Epoch 6/10
127657/127657 [==============================] - 58s 453us/step - loss: 0.0298 - acc: 0.9879 - val_loss: 0.0445 - val_acc: 0.9835
Epoch 7/10
127657/127657 [==============================] - 59s 460us/step - loss: 0.0281 - acc: 0.9886 - val_loss: 0.0460 - val_acc: 0.9833
Epoch 8/10
127657/127657 [==============================] - 59s 461us/step - loss: 0.0265 - acc: 0.9892 - val_loss: 0.0473 - val_acc: 0.9834
Epoch 9/10
127657/127657 [==============================] - 59s 460us/step - loss: 0.0253 - acc: 0.9897 - val_loss: 0.0498 - val_acc: 0.9834
Epoch 10/10
127657/127657 [==============================] - 59s 459us/step - loss: 0.0243 - acc: 0.9902 - val_loss: 0.0496 - val_acc: 0.9821
-------------------------------
0 0.08907060847953205 0.9657581891446441
1 0.02190176968998027 0.9905809952936311
2 0.044300770626223875 0.98237148354024